**Notebook Description: Query Retrieval Evaluation and Optimization**

This notebook evaluates the effectiveness of article retrieval using two different query methods: the original query and an extended query with added relevant terms. By comparing the relevance of articles retrieved through each approach, we assess whether query expansion enhances the quality of results. The main comparison tool is based on model evaluations: the combined opinions of three language models on the relevance of each article’s abstract to the query. We use the models’ scores, their averaged results, and cosine similarity to measure and optimize retrieval quality.

In [ ]:
#imports
!pip install openai==0.28
!pip install pinecone cohere openai sentence_transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics import ndcg_score
import google.generativeai as genai
import pinecone
import openai
import cohere
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from pinecone import Pinecone, ServerlessSpec
import os


In [ ]:
# Set up Pinecone
pc = Pinecone(api_key='YOUR-API-KEY')
index = pc.Index("document-embeddings")

In [ ]:
# Set up OpenAI, Gemini and Cohere API keys
openai.api_key = 'YOUR-API-KEY'
co = cohere.Client('YOUR-API-KEY')
genai.configure(api_key='YOUR-API-KEY')


In [ ]:
# Initialize SentenceTransformer model for embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")

# Function to encode text using SentenceTransformer
def encode_text(text):
    embeddings = model.encode(text)
    return embeddings

In [ ]:
# Retrieve top K articles from Pinecone
def retrieve_articles(query, top_k=5):
    query_embedding = encode_text(query)
    response = index.query(vector=query_embedding.tolist(), top_k=top_k, include_metadata=True, include_values=True)
    articles = [(match['metadata']['title'], match['metadata']['abstract']) for match in response['matches']]
    return articles

# Evaluate articles using GPT, Cohere, and Gemini
def evaluate_articles(query, articles):
    gpt_scores, cohere_scores, gemini_scores = [], [], []
    for title, abstract in articles:
        combined_text = f"Title: {title}\nAbstract: {abstract}"

        # OpenAI GPT Score
        gpt_score = call_gpt_api(query, combined_text)
        gpt_scores.append(gpt_score)

        # Cohere Score
        cohere_score = call_cohere_api(query, combined_text)
        cohere_scores.append(cohere_score)

        # Gemini Score
        gemini_score = call_gemini_api(query, combined_text)
        gemini_scores.append(gemini_score)

    avg_scores = [(g + c + a) / 3 for g, c, a in zip(gpt_scores, cohere_scores, gemini_scores)]
    return avg_scores

# GPT API call
def call_gpt_api(query, abstract):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a relevance evaluator for scientific articles. Reply with only a single number between 1 and 10."},
            {"role": "user", "content": f"Question: {query}\nArticle Abstract: {abstract}\nRate the relevance from 1 to 10. Only provide the number as an answer."}
        ]
    )
    score_text = response['choices'][0]['message']['content']
    return float(score_text.strip())


# Cohere API call
def call_cohere_api(query, abstract):
    response = co.generate(
        model="command-xlarge-nightly",
        prompt=f"You are a relevance evaluator for scientific articles. Question: {query}\nArticle Abstract: {abstract}\nRate the relevance from 1 to 10. Only provide the number as an answer.",
        max_tokens=10
    )
    return float(response.generations[0].text.strip())


# Gemini API call
def call_gemini_api(query, abstract):
    model = genai.GenerativeModel("gemini-1.5-flash")
    prompt_text = f"You are a relevance evaluator for scientific articles. Question: {query}\nArticle Abstract: {abstract}\nRate the relevance from 1 to 10. Only provide the number as an answer."
    response = model.generate_content(prompt_text)
    return float(response.text)


# Query expansion using GPT
def expand_query(query):
    # Generate expanded terms related to the original query using GPT
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert in data science and information retrieval."},
            {"role": "user", "content": f"Provide additional keywords or phrases that could be incorporated into the following query to refine or broaden it, but do not answer the question. Return a concise list of relevant terms or phrases only. Query: '{query}'"}
        ]
    )
    expanded_terms = response['choices'][0]['message']['content'].strip()
    expanded_query = f"{query} {expanded_terms}"  # Combine original query with expanded terms
    return expanded_query



**Creating a table to compare the results:**

In [ ]:

# Define the CSV file name
csv_file = "query_expansion_results.csv"

# Check if CSV exists and load it if it does
if os.path.exists(csv_file):
    results_df = pd.read_csv(csv_file)
else:
    # Initialize an empty DataFrame if the CSV does not exist
    columns = [
        "query", "expanded_query",
        "title", "abstract", "model_score", "cosine_similarity",
        "source_query", "avg_original_score", "avg_expanded_score",
        "overlap_count"
    ]
    results_df = pd.DataFrame(columns=columns)

# Function to calculate cosine similarity
def calculate_cosine_similarity(query_embedding, abstract_embedding):
    return cosine_similarity([query_embedding], [abstract_embedding])[0][0]

# Test each question with and without query expansion
def test_query_expansion(questions, top_k=5):
    global results_df

    for query in questions:
        # Original query retrieval and scoring
        print(f"Running retrieval for original query: {query}")
        original_articles = retrieve_articles(query, top_k)
        original_scores = evaluate_articles(query, original_articles)
        avg_original_score = np.mean(original_scores)
        query_embedding = encode_text(query)  # Embedding of the original query

        # Expanded query retrieval and scoring
        expanded_query = expand_query(query)
        expanded_articles = retrieve_articles(expanded_query, top_k)
        expanded_scores = evaluate_articles(query, expanded_articles)
        avg_expanded_score = np.mean(expanded_scores)

        # Track seen titles to manage overlaps and label source query
        original_titles = {title for title, _ in original_articles}
        all_articles = {}

        # Process original query articles
        for i, (title, abstract) in enumerate(original_articles):
            abstract_embedding = encode_text(abstract)
            cosine_sim = calculate_cosine_similarity(query_embedding, abstract_embedding)
            all_articles[title] = {
                "query": query,
                "expanded_query": expanded_query,
                "title": title,
                "abstract": abstract,
                "model_score": original_scores[i],
                "cosine_similarity": cosine_sim,
                "source_query": "original",
            }

        # Process expanded query articles, check for overlap
        for i, (title, abstract) in enumerate(expanded_articles):
            abstract_embedding = encode_text(abstract)
            cosine_sim = calculate_cosine_similarity(query_embedding, abstract_embedding)
            if title in all_articles:
                # Mark as overlap if already seen in original query results
                all_articles[title]["source_query"] = "overlap"
            else:
                # New article from expanded query
                all_articles[title] = {
                    "query": query,
                    "expanded_query": expanded_query,
                    "title": title,
                    "abstract": abstract,
                    "model_score": expanded_scores[i],
                    "cosine_similarity": cosine_sim,
                    "source_query": "expanded",
                }

        # Add results to DataFrame
        for article_data in all_articles.values():
            new_row = pd.DataFrame([article_data])
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        # Calculate overlap count
        overlap_count = sum(1 for article in all_articles.values() if article["source_query"] == "overlap")

        # Save intermediate results to the CSV file
        # results_df["avg_original_score"] = avg_original_score
        # results_df["avg_expanded_score"] = avg_expanded_score
        # results_df["overlap_count"] = overlap_count
        results_df.to_csv(csv_file, index=False)
        print(f"Results saved to CSV after query: {query}")

    return results_df




**The list of questions we evaluated and the comparison run:**

In [ ]:

# List of questions to run one at a time
questions = [
    "What are the latest techniques for explainability in deep neural networks?",
    "How is transfer learning being applied in medical imaging analysis?",
    "What are the advancements in handling class imbalance in supervised learning?",
    "How can reinforcement learning improve autonomous vehicle navigation?",
    "What methods exist for detecting outliers in financial time series?",
    "What are recent innovations in synthetic data generation for NLP?",
    "How is unsupervised learning being used in anomaly detection?",
    "What are the latest approaches to causal inference in data science?",
    "How is AI being used to optimize supply chain management?",
    "What are novel techniques for feature engineering in time series data?",
    "How can GANs (Generative Adversarial Networks) improve image resolution?",
    "What role does data augmentation play in small dataset training?",
    "How is federated learning enhancing privacy in healthcare applications?",
    "What are effective methods for hyperparameter tuning in deep learning?",
    "How can active learning reduce labeling costs in supervised learning?",
    "What are the advancements in dimensionality reduction techniques for visualization?",
    "How is NLP being used to analyze sentiment in financial news?",
    "What methods are effective for handling missing data in datasets?",
    "How is transfer learning improving outcomes in drug discovery research?",
    "What are the latest machine learning techniques for predictive maintenance?"
]

# Run the function with each question, waiting 30 seconds between runs
for question in questions:
    print(f"Running query expansion for question: {question}")
    results_df = test_query_expansion([question])  # Call function with a single question in a list
    print(f"Completed query for: {question}")
    print("Waiting 35 seconds before the next run...")
    time.sleep(35)  # Wait 35 seconds before moving to the next question

print("All queries completed.")


**Displaying the results according to the value of the model score:**

In [ ]:

results_df = pd.read_csv("query_expansion_results.csv")

# Function to display all results for a specific query in three sorting orders
def display_all_results_for_query(query):
    # Filter results for the specified query
    query_results = results_df[results_df["query"] == query].copy()

    # Ensure that the average score column is computed
    query_results["average_score"] = query_results[["cosine_similarity", "model_score"]].mean(axis=1)

    # Sort by Model Score
    sorted_by_model = query_results.sort_values(by="model_score", ascending=False).reset_index(drop=True)
    print(f"\n--- All Results for Query '{query}' Sorted by Model Score ---")
    display(sorted_by_model[["title", "abstract", "source_query", "cosine_similarity", "model_score", "average_score"]])

for question in questions:
  display_all_results_for_query(question)



--- All Results for Query 'What are the latest techniques for explainability in deep neural networks?' Sorted by Model Score ---


,title,abstract,source_query,cosine_similarity,model_score,average_score
0,Explaining Deep Neural Networks and Beyond: A ...,With the broader and highly successful usage...,overlap,0.725217,9.333333,5.029275
1,Gradient based Feature Attribution in Explaina...,The surge in black-box AI models has prompte...,expanded,0.716860,9.000000,4.858430
2,Robust Explainability: A Tutorial on Gradient-...,"With the rise of deep neural networks, the c...",overlap,0.785365,8.666667,4.726016
3,Explaining Deep Neural Networks,Deep neural networks are becoming more and m...,original,0.739407,7.333333,4.036370
4,How Good is your Explanation? Algorithmic Stab...,A plethora of methods have been proposed to ...,original,0.713152,7.000000,3.856576
5,Explainable Artificial Intelligence (XAI): Con...,"In the last years, Artificial Intelligence (...",expanded,0.673859,6.666667,3.670263
6,Towards a Unified Evaluation of Explanation Me...,This paper proposes a set of criteria to eva...,original,0.770281,6.333333,3.551807
7,Do Explanations Reflect Decisions? A Machine-c...,There has been a significant surge of intere...,expanded,0.703462,6.333333,3.518398



--- All Results for Query 'How is transfer learning being applied in medical imaging analysis?' Sorted by Model Score ---


,title,abstract,source_query,cosine_similarity,model_score,average_score
0,What Makes Transfer Learning Work For Medical ...,Transfer learning is a standard technique to...,overlap,0.720322,9.333333,5.026828
1,"Deep transfer learning for detecting Covid-19,...",Chest X-rays remains to be the most common i...,expanded,0.627158,9.333333,4.980246
2,MedNet: Pre-trained Convolutional Neural Netwo...,Deep Learning (DL) requires a large amount o...,expanded,0.578660,9.333333,4.955997
3,Supervised Transfer Learning at Scale for Medi...,Transfer learning is a standard technique to...,original,0.683767,9.000000,4.841883
4,Modality-bridge Transfer Learning for Medical ...,This paper presents a new approach of transf...,original,0.694051,9.000000,4.847025
5,Towards Automated Melanoma Screening: Explorin...,Deep learning is the current bet for image c...,expanded,0.626437,8.666667,4.646552
6,Transfusion: Understanding Transfer Learning f...,Transfer learning from natural image dataset...,expanded,0.587944,8.333333,4.460639
7,Exploring connections of spectral analysis and...,"In this paper, we use spectral analysis to i...",original,0.604168,7.666667,4.135418
8,Visual Transfer Learning: Informal Introductio...,Transfer learning techniques are important t...,original,0.633252,3.333333,1.983293



--- All Results for Query 'What are the advancements in handling class imbalance in supervised learning?' Sorted by Model Score ---


...

,title,abstract,source_query,cosine_similarity,model_score,average_score
0,Identification and Estimation for Nonignorable...,We consider the task of identifying and esti...,original,0.623152,9.333333,4.978243
1,On the Performance of Imputation Techniques fo...,Missing values or data is one popular charac...,expanded,0.560545,9.333333,4.946939
2,Review for Handling Missing Data with special ...,Missing data poses a significant challenge i...,original,0.718114,9.000000,4.859057
3,Missing Data Imputation using Optimal Transport,Missing data is a crucial issue when applyin...,overlap,0.655189,9.000000,4.827595
4,A computational study on imputation methods fo...,Data acquisition and recording in the form o...,overlap,0.649366,9.000000,4.824683
5,Explainable Data Imputation using Constraints,Data values in a dataset can be missing or a...,expanded,0.687053,9.000000,4.843527
6,The Missing Indicator Method: From Low to High...,Missing data is common in applied data scien...,original,0.661660,8.666667,4.664163
7,Performance comparison of State-of-the-art Mis...,Decision making from data involves identifyi...,expanded,0.591255,8.666667,4.628961



--- All Results for Query 'How is transfer learning improving outcomes in drug discovery research?' Sorted by Model Score ---


,title,abstract,source_query,cosine_similarity,model_score,average_score
0,An Integrated Transfer Learning and Multitask ...,Background: Pharmacokinetic evaluation is on...,overlap,0.546755,9.000000,4.773377
1,Ensemble Transfer Learning for the Prediction ...,Transfer learning has been shown to be effec...,expanded,0.513867,9.000000,4.756933
2,Improving Compound Activity Classification via...,Recent advances in molecular machine learnin...,expanded,0.564232,9.000000,4.782116
3,What Makes Transfer Learning Work For Medical ...,Transfer learning is a standard technique to...,original,0.596870,6.333333,3.465101
4,Transfer Learning Bayesian Optimization to Des...,With the rise in engineered biomolecular dev...,original,0.585448,6.333333,3.459391
5,Is Meta-training Really Necessary for Molecula...,Few-shot learning has recently attracted sig...,original,0.576349,6.333333,3.454841
6,Artificial Intelligence for Drug Discovery: Ar...,Drug discovery is adapting to novel technolo...,expanded,0.547787,5.666667,3.107227
7,Research Frontiers in Transfer Learning -- a s...,"Humans can learn from very few samples, demo...",original,0.506291,4.000000,2.253146
8,Applications of artificial intelligence in dru...,The US Food and Drug Administration (FDA) ha...,expanded,0.484070,4.000000,2.242035



--- All Results for Query 'What are the latest machine learning techniques for predictive maintenance?' Sorted by Model Score ---


,title,abstract,source_query,cosine_similarity,model_score,average_score
0,Deep learning models for predictive maintenanc...,Given the growing amount of industrial data ...,expanded,0.689229,9.333333,5.011281
1,Revolutionizing System Reliability: The Role o...,The landscape of maintenance in distributed ...,overlap,0.721966,9.000000,4.860983
2,Comprehensive Study Of Predictive Maintenance ...,"In today's technology-driven era, the impera...",overlap,0.687637,8.000000,4.343818
3,Predictive Maintenance using Machine Learning,"Predictive maintenance (PdM) is a concept, w...",overlap,0.692235,6.666667,3.679451
4,A Benchmark dataset for predictive maintenance,"The paper describes the MetroPT data set, an...",overlap,0.749577,5.666667,3.208122
5,An Economic Perspective on Predictive Maintena...,This paper provides an economic perspective ...,original,0.724792,4.666667,2.695729


**Statistics of the results:**

In [ ]:

results_df = pd.read_csv("query_expansion_results.csv")

# Initialize a list to store counts of expanded articles per query
expanded_counts = []

# List of queries to analyze
queries = results_df["query"].unique()

for query in queries:
    # Filter results for the specified query
    query_results = results_df[results_df["query"] == query].copy()

    # Sort by Model Score and select the top 5 articles
    top_5_by_model = query_results.sort_values(by="model_score", ascending=False).head(5)

    # Count the number of expanded articles in the top 5
    expanded_count = (top_5_by_model['source_query'] == 'expanded').sum()
    expanded_counts.append(expanded_count)

    # Display the top 5 results sorted by model score for the query
    print(f"\n--- Top 5 Articles for Query '{query}' Sorted by Model Score ---")
    print(f"Number of expanded articles in top 5 by model score: {expanded_count}\n")

# Calculate and print the average count of expanded articles in the top 5 across all queries
average_expanded_count = sum(expanded_counts) / len(expanded_counts)
print(f"Average number of expanded articles in the top 5 across all queries: {average_expanded_count}")


In [ ]:

results_df = pd.read_csv("query_expansion_results.csv")

# Initialize a list to store counts of expanded articles per query
expanded_counts = []

# List of queries to analyze
queries = results_df["query"].unique()

for query in queries:
    # Filter results for the specified query
    query_results = results_df[results_df["query"] == query].copy()

    # Sort by Model Score and select the top 5 articles
    top_5_by_model = query_results.sort_values(by="model_score", ascending=False).head(5)

    # Count the number of expanded articles in the top 5
    expanded_count = (top_5_by_model['source_query'] == 'expanded').sum()
    expanded_counts.append(expanded_count)

    # Display the top 5 results sorted by model score for the query
    print(f"\n--- Top 5 Articles for Query '{query}' Sorted by Model Score ---")
    print(f"Number of expanded articles in top 5 by model score: {expanded_count}\n")

# Calculate and print the average count of expanded articles in the top 5 across all queries
average_expanded_count = sum(expanded_counts) / len(expanded_counts)
print(f"Average number of expanded articles in the top 5 across all queries: {average_expanded_count}")



--- Top 5 Articles for Query 'What are the latest techniques for explainability in deep neural networks?' Sorted by Model Score ---
Number of expanded articles in top 5 by model score: 1


--- Top 5 Articles for Query 'How is transfer learning being applied in medical imaging analysis?' Sorted by Model Score ---
Number of expanded articles in top 5 by model score: 2


--- Top 5 Articles for Query 'What are the advancements in handling class imbalance in supervised learning?' Sorted by Model Score ---
Number of expanded articles in top 5 by model score: 1


--- Top 5 Articles for Query 'How can reinforcement learning improve autonomous vehicle navigation?' Sorted by Model Score ---
Number of expanded articles in top 5 by model score: 2


--- Top 5 Articles for Query 'What methods exist for detecting outliers in financial time series?' Sorted by Model Score ---
Number of expanded articles in top 5 by model score: 0


--- Top 5 Articles for Query 'What are recent innovations in synthetic